In [44]:
import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
import spacy
import math
import os
from src.utils import *

os.chdir('/Users/ismailakrout/Desktop/python/statapp')

pd.set_option("display.max_columns", None)


In [45]:
dtypes = {
    'party.facts.id' : str,
    'date': object,
    'agenda': object,
    'speechnumber': int,
    'speaker': object,
    'party': object,
    'party.facts.id': object,
    'chair': bool,
    'terms': int,
    'text': object,
}

In [46]:
df = read_input('00. input/Corp_HouseOfCommons_V2_2010-2.csv', encod='ISO-8859-1', dtype_values=dtypes)
df.drop(columns=['Unnamed: 0', 'iso3country', 'parliament', 'party.facts.id', 'speechnumber', 'chair', 'terms', 'date'], inplace=True)
df.rename(columns=
    {'speaker': 'Speaker'},
    inplace=True
)

In [47]:
# pour faire des essais rapides on réduit la taille du DataFrame

df = df.sample(frac=1).reset_index(drop=True)
df = df.head(1000)


In [48]:
# on ne garde que Labor et Conservative
df = df.loc[df['party'].isin(['Lab', 'Con'])]

## Data processing

On va tout d'abord créer une fonction qui va appliquer le même traitement aux speeches qui a été appliqué aux textes des journaux : 
- tokenization 
- nettoyage de la ponctuation et des nombres 
- stemming des mots 
- nettoyage des stopwords à l'aide de la même liste que pour les journaux
- remise des mêmes stopwords significatifs que pour les journaux 

In [49]:
from spacy.lang.en import English
nlp = spacy.load('en_core_web_sm')
nlp.disable_pipes(["tagger", "parser"])
import string
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')

On utilise le SnowballStemmer.

On importe la liste des stopwords :

In [50]:
df_stopwords = pd.read_csv("00. input/sw1k.csv",
                names=['word', 'frequency', 'presence', 'doc_size_sum', 'type'],
                encoding='latin-1').drop(index=0)

On a donc un dataframe des stopwords. On en tire une liste simple des stopwords, où l'on remet cependant des stopwords jugés significatifs, comme expliqué pour le traitement des journaux :

In [51]:
stopwords = df_stopwords['word'].unique()
putback = ['prime', 'officials', 'security', 'news', 'working', 'games', 'jobs', 'campaign', 'services',
'civil', 'economic', 'information', 'political', 'election', 'court', 'office', 'vote', 'trump', 'control', 'job', 'price',
'donald trump', 'chinese', 'problems', 'concerns', 'minister', 'nation', 'policy', 'data', 'indian', 'congress',
'president', 'network', 'american', 'accused', 'government', 'money', 'investigation',
'facebook', 'success', 'prices', 'twitter', 'book', 'politics',
'justice', 'claims', 'russia', 'law', 'technology', 'content', 'union', 'european', 'workers']

stopwords = list(set(stopwords) - set(putback))


On utilise la librairie SpaCy comme pour le traitement des journaux, et on enlève les nombres et la ponctuation avec la méthode translate. 

In [52]:
def clean(text):
  text = str(text).lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = text.translate(str.maketrans('', '', string.digits)) 
  doc = nlp(text)
  l = []
  for word in doc :
    word = str(word)
    l.append(stemmer.stem(word))
    if l[-1] == ' ' or l[-1] in stopwords: 
      l.pop()
  return l

In [53]:
df['agenda'] = df['agenda'].apply(clean)

/Users/ismailakrout/opt/anaconda3/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


On peut ensuite appliquer ce nettoyage à nos speeches, et aux sujets de ceux-ci, directement dans le dataframe. 

Maintenant, chaque speech est transformé en une liste de mots significatifs. 

On voit par exemple que 24 speeches de l'échantillon ont "data" répertorié comme sujet. 

# **Construction du test**

On va tout d'abord essayer de filtrer les speeches qui sont dans le "thème" des nouvelles technologies, big tech, etc. Pour ça, on se créé une liste de mots clés à partir des mots clés utilisés pour les journaux, et on ne garde que les speeches qui sont concernés par ces sujets. On commence par faire une liste assez grossière. 

In [54]:
technology=['technology','innovat','computer','high tech|high-tech','science','engineering']
consumer_protection=['privacy','data leak','leak','fake news',' safety','decept','defective','hack']
firms=['google','alphabet','apple','facebook','meta','amazon','microsoft']
products=['chrome','incognito','youtube','nexus','pixel','google drive','gmail','glass','street view','buzz','fitbit',
 'maps', 'doodle','play','translate','search', 'google news','nest hub','xl','nest','chromecast','stadia','hub',
 'marshmallow','lollipop','cloud','waymo','earth','engine',

'apple pay','apple watch','iphone','ipad','ipod','iwatch','macbook','macbook pro', 'macbook air','mac',
 'imac','airpods','ios','siri','icloud','apple tv','apple music','app store', 'safari','x','app','apple store',
 'xr', 'xs', 'se','iphones','itunes','ibook','plus','pro','max','mini','os','airtag','airtags','arcade','homepod',
'keynote','ipados','id','foxconn','facetime','beat','stalk',

'messenger','instagram', 'whatsapp','page','feed','oculus',

'prime', 'kindle',
'publishing','amazon prime','amazon drive','amazon video','amazon business','amazon web service',
'amazon cloud', 'alexa','echo dot','echo','dot', 'delivery', 'amazon uk','unlimited', 'episode','foods','grocery', 
'grand tour','grand','tour','viking', 'vikings','argo','argos','macmillan', 'dvd','clarkson','lord','ring','hair','skin','vacuum',
'pre','beer','drake','spark','kart','dog','twitch','cat','xo','matthew','stafford','ratchet','clank',
'swagway','album','mouse','showbiz','beauty','guardian','batman','arkham','gc','hair','skin','shirt',
'lovefilm','mirzapur','cast','audio','drama','movie','jack ryan','actor','character','lucifer','outlander',
'premier','super mario','sky','channel','voyage',


'windows','window','xp','surface','xbox','studio','microsoft office', 'office','word','cortana', 'surface pro','teams',  'playstation',
'microsoft edge', 'edge', 'gear','outlook','halo','skype','kinect','internet explorer','explorer','ie','bing','xcloud','hololens',
'forza','ori','scarlett','scorpio','wordperfect','valhalla','onedrive','games gold','lumia','azure',
'assassin creed','assassin','creed','minecraft','yammer','warcraft','tay']



ceos=['sundar','pichai','eric','schmidt','steve jobs','tim cook','mark zuckerberg','andy jassy','jeff','bezos','satya', 'nadella','bill gates',
      'gates','steve job','steve','tim', 'cook','zuckerberg','ceo','tim cook ','steve ballmer','ballmer','elop',
      'schiller','fadell','phil spencer','spencer','mcspirit','sandberg','paul','allen','larry hryb','hryb']
      

types=['tablet','mobile', 'laptop', 'pc', 'computer', 'desktop','smartphone', 'smartwatch', 'search engine', 'software','hardware',
               'machine', 'browser','ebook', 'book',  'reader',  'console', 'headphone', 'earbud','bud','store','music',
              'gaming', 'operating','streaming','title','chatbot']



competitor=['samsung', 'galaxy',  'twitter','tiktok', 'switch','sony', 'asos', 'activision blizzard', 'activision','blizzard',
            'nintendo','snes', 'netflix','android','yahoo','nokia','huawei','motorola','htc','blackberry','oppo','oneplus','rim','symbian','bbc','morrison','spotify'] 


configue=['device','feature','battery','screen','sound','gb','g','k','mm','chip','processor','design','display','touch','ram',
          'inch','keyboard','camera','handset','speaker','button','touchscreen','storage', 'data']


celebrity=['dubost','neymar','amanda','beyonce','blur','richard','hammond','ranj','jeremy clarkson', 'jeremy','momoa',
           'jared','aniston','smith','kim','tony','tom','sophie','oasis','trio','sharon','betty','raoul','moat','lauren','andrew',
           'samuel gibbs','samuel','gibbs','van','gaal']

topics = celebrity + configue + competitor + types + ceos + products + firms + consumer_protection + technology

On stem les topics : 

In [55]:
string_of_topics = ' '.join(topics)
string_of_topics = stemmer.stem(string_of_topics)
stem_topics = string_of_topics.split(' ')

In [56]:
# c'est trop nul mais je n'arrive pas à l'améliorer 

def techno_filter(list):
  for k in range(len(list)):
    if list[k] in stem_topics :
      return True
  return False

In [57]:
df['agenda'] = df['agenda'].apply(techno_filter)

In [60]:
S = df.loc[df['agenda']].shape[0]
S

84

On voit que sur les 5000 premiers speeches, seuls concernent les topics définis. C'est peu, mais plutôt logique vu que ces speeches ont été faits en 1988 environ. 

On ne garde que ceux-là :

In [22]:
df = df.loc[df['agenda']]
df.drop(columns=['agenda'], inplace=True)

/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_20327/2965606736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['agenda'], inplace=True)


In [23]:
df['text'] = df['text'].apply(clean)

/Users/ismailakrout/opt/anaconda3/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_20327/118106318.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(clean)


In [24]:
# On fait le groupby() par speaker et party par la suite pour ne pas dépasser les 1 000 000 carc 
df = df.groupby(by=['Speaker', 'party']).sum().reset_index()

On va repérer les 100 mots (puis 500 quand on aura les speeches en entier) qui sont le plus utilisés par chaque parti, et noter leur fréquence d'apparition. 

In [25]:
aux = df[['party', 'text']]

In [26]:
aux.groupby(by=['party']).sum()

,text
party,
Con,"[hon, prime, minist, deserv, credit, syria, re..."
Lab,"[hon, lilli, bridg, depot, element, earl, cour..."


In [27]:
def words_of_party(df, text):
  l = []
  for i in df.index: 
    if df['party'][i] == text :
      l = l + df['text'][i]
  return l

In [28]:
words_of_Con = words_of_party(df, 'Con')
words_of_Lab = words_of_party(df, 'Lab')
# words_of_LibDem = words_of_party(df, 'LibDem')
# words_of_SDLP = words_of_party(df, 'SDLP')

In [29]:
def words_frequencies(list):
  words = []
  freqs = []
  for word in list: 
    if word not in words : 
      words.append(word)
      freqs.append(1)
    else :
      k = words.index(word)
      freqs[k] += 1
  d = {'words': words, 'freqs': freqs}
  words_freqs =  pd.DataFrame(data=d)
  return words_freqs

In [30]:
words_frequencies(words_of_Con)

,words,freqs
0,hon,40
1,prime,6
2,minist,17
3,deserv,2
4,credit,2
...,...,...
555,origin,1
556,custom,1
557,independ,1
558,dealsgood,1


On a tous les mots utilisés par le parti Conservateur sur les sujets définis, ainsi que leurs fréquences d'apparition. On va retenir les fréquences avec lesquelles chaques mots sont utilisés, par chaque parti, afin de déterminer le chi2 de chaque mot. On se restreint pour l'instant aux partis Con et Lab, fortement majoritaires. 

In [31]:
df_freqs = pd.merge(words_frequencies(words_of_Con), words_frequencies(words_of_Lab), on="words", how='outer')

In [32]:
c = 0
for i in range(len(df_freqs['words'])):
  if df_freqs['freqs_x'][i] > 0 and df_freqs['freqs_y'][i] >0 :
    c += 1

In [33]:
c

160

Il y a donc c mots utilisés par les deux partis. 

In [34]:
df_freqs

,words,freqs_x,freqs_y
0,hon,40.0,25.0
1,prime,6.0,15.0
2,minist,17.0,18.0
3,deserv,2.0,NaN
4,credit,2.0,1.0
...,...,...,...
1086,dent,NaN,1.0
1087,luxuri,NaN,1.0
1088,sweetheart,NaN,1.0
1089,paltri,NaN,1.0


In [35]:
# Correction du test du Qi 2 

df_freqs.loc[df_freqs['freqs_x'].notna(), 'fp_Con'] = df_freqs.loc[df_freqs['freqs_x'].notna(), 'freqs_x']
df_freqs['fp_Con'] = df_freqs['fp_Con'].fillna(0)

df_freqs.loc[df_freqs['freqs_y'].notna(), 'fp_Lab'] = df_freqs.loc[df_freqs['freqs_y'].notna(), 'freqs_y']
df_freqs['fp_Lab'] = df_freqs['fp_Lab'].fillna(0)

s_con = df_freqs['fp_Con'].sum()
df_freqs['not_fp_Con'] = s_con - df_freqs['fp_Con']

s_lab = df_freqs['fp_Lab'].sum()
df_freqs['not_fp_Lab'] = s_lab - df_freqs['fp_Lab']

df_freqs['chi_2'] = ((df_freqs['fp_Con'] * df_freqs['not_fp_Con'] - df_freqs['fp_Lab'] * df_freqs['not_fp_Lab'])**2) / ((df_freqs['fp_Con']+df_freqs['fp_Lab']) * (df_freqs['not_fp_Lab']+df_freqs['fp_Lab']) * (df_freqs['not_fp_Con']+df_freqs['fp_Con']) * (df_freqs['not_fp_Con']+df_freqs['not_fp_Lab']))

df_freqs.drop(columns=['fp_Con', 'fp_Lab', 'not_fp_Con', 'not_fp_Lab'], inplace=True)

In [36]:
df_freqs

,words,freqs_x,freqs_y,chi_2
0,hon,40.0,25.0,0.000311
1,prime,6.0,15.0,0.002945
2,minist,17.0,18.0,0.000362
3,deserv,2.0,NaN,0.000743
4,credit,2.0,1.0,0.000066
...,...,...,...,...
1086,dent,NaN,1.0,0.000598
1087,luxuri,NaN,1.0,0.000598
1088,sweetheart,NaN,1.0,0.000598
1089,paltri,NaN,1.0,0.000598


On a donc ici chaque mot utilisé par les parti sur le sujet délimité, avec le chi2 correspondant. On va se limiter aux 500 premiers mots :

In [37]:
df_freqs = df_freqs.sort_values(by=['chi_2'], ascending=False)

In [38]:
df_freqs = df_freqs.head(500)

In [39]:
df_freqs

,words,freqs_x,freqs_y,chi_2
843,amend,NaN,15.0,0.008824
655,internet,NaN,10.0,0.005919
896,regul,NaN,9.0,0.005334
884,dog,NaN,9.0,0.005334
673,ofcom,NaN,9.0,0.005334
...,...,...,...,...
913,mix,NaN,1.0,0.000598
914,slower,NaN,1.0,0.000598
915,english,NaN,1.0,0.000598
916,franc,NaN,1.0,0.000598


In [40]:
# là on obtient la liste des 500 mots les plus importants 
lili = df_freqs['words'].unique()

# On forme le tableau 

In [41]:
lili

array(['amend', 'internet', 'regul', 'dog', 'ofcom', 'balanc', 'leveson',
       'cover', 'privaci', 'radio', 'educ', 'music', 'claus', 'copyright',
       'kingdom', 'audit', 'gentleman', 'prime', 'money', 'european',
       'breed', 'ipso', 'bromwich', 'inform', 'benefit', 'data',
       'newcastl', 'convent', 'materi', 'communic', 'accept', 'miner',
       'channel', 'bath', 'adopt', 'intellectu', 'prepar', 'speech',
       'sector', 'pension', 'defend', 'union', 'govern', 'patient',
       'creativ', 'miss', 'common', 'impos', 'persuad', 'steal',
       'deliber', 'file', 'tabl', 'send', 'mention', 'wiley', 'owner',
       'broadcast', 'activ', 'complex', 'scheme', 'inquiri', 'selfregul',
       'surpris', 'procedur', 'therefor', 'googl', 'negoti', 'mental',
       'secur', 'nhs', 'consid', 'debat', 'agre', 'wish', 'draft', 'pii',
       'musician', 'walk', 'newspap', 'lack', 'vest', 'extraordinarili',
       'orphan', 'softwar', 'brick', 'enrich', 'joe', 'precept',
       'exhaust

In [42]:
'''
# La c'est bien codé
def count_liste(lili, mot):
    return lili.count(mot)
for word in lili:
    df[f'{word}'] = df['text'].apply(count_liste, args=(word,))
'''

/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_20327/925462558.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{word}'] = df['text'].apply(count_liste, args=(word,))
/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_20327/925462558.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{word}'] = df['text'].apply(count_liste, args=(word,))
/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_20327/925462558.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

In [230]:
# tres mal codé
for word in lili:
    def count_liste(lili):
        return lili.count(word)
    df[f'{word}'] = df['text'].apply(count_liste)


/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_14956/451958271.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{word}'] = df['text'].apply(count_liste)
/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_14956/451958271.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{word}'] = df['text'].apply(count_liste)
/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_14956/451958271.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` m

In [231]:
df['somme'] = df.sum(axis=0)

/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_14956/1347020148.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['somme'] = df.sum(axis=0)


In [232]:
a = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]])

In [233]:
a.sum(axis=0)

array([12, 15, 18])

In [234]:
df['somme'] = df.sum(axis=1)
df.loc[df['somme'] == 0, 'somme'] = 1

/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_14956/642274951.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['somme'] = df.sum(axis=1)


In [235]:
df['somme']

0      1030
1       225
2       444
3       100
4        94
       ... 
860     382
861    1517
862    2709
863     676
864     798
Name: somme, Length: 865, dtype: int64

In [236]:
for word in lili:
    df[f'{word}'] = df[f'{word}'] / df['somme'] 

In [237]:
df

,Speaker,party,text,hon,gentleman,govern,ladi,absolut,european,labour,agre,deliv,constitu,uk,commit,â£,recognis,money,union,debat,welcom,fund,amend,discuss,improv,nhs,kingdom,protect,law,secur,reform,consid,encourag,requir,benefit,eu,lord,bbc,job,sector,parliament,duti,oper,concern,review,technolog,legisl,regul,vote,effect,spend,measur,introduc,data,provis,inde,appli,britain,offenc,specif,addit,programm,propos,individu,wish,success,elect,claus,econom,congratul,obvious,shall,grate,inform,learn,progress,patient,enabl,agreement,proper,subject,awar,refer,england,therefor,organis,polit,independ,achiev,complet,court,respect,northern,ireland,excel,approach,exist,vital,delight,apprenticeship,shadow,sort,activ,practic,colleagu,perhap,g,exact,speaker,crime,beer,educ,extrem,scotland,commend,employ,hear,mention,negoti,assur,campaign,referendum,promot,suggest,yes,carri,polici,design,contribut,tribut,necessari,reassur,crimin,extra,serv,rural,fair,appropri,accept,arrang,scheme,greater,pub,enterpris,dog,innov,engin,scottish,control,chariti,fulli,respond,engag,product,conduct,listen,basi,trust,initi,reduc,common,train,scienc,deficit,welfar,confid,commiss,defenc,regim,maintain,investig,strategi,circumst,consum,standard,principl,consult,purpos,ministeri,rememb,prosecut,determin,object,mobil,argument,broadband,consider,longterm,midland,presid,speech,figur,resid,proud,proceed,generat,establish,border,coverag,cover,gas,skill,environ,export,unemploy,prison,repres,reflect,wind,incred,infrastructur,implement,safe,abil,green,relev,sentenc,cornwal,highlight,solut,taxpay,natur,competit,remind,answer,balanc,freedom,framework,pass,citizen,extend,simpli,regist,syria,attract,flexibl,draft,partnership,intervent,remov,sir,advic,rout,produc,remark,connect,communic,appoint,tackl,network,identifi,judg,perfect,penalti,intent,intend,link,commission,prioriti,chancellor,arm,cultur,drug,wale,applic,electr,word,section,outcom,custom,justic,inquiri,enforc,essenti,procedur,stalk,owner,price,statutori,treati,aim,terrorist,throughout,cancer,send,transpar,guidanc,sensibl,devolv,doubt,highway,recommend,choic,notic,urg,whom,approv,vehicl,regular,prepar,paper,elector,exercis,assist,strike,enjoy,inherit,safeguard,precis,sit,capabl,save,constitut,pupil,partner,treat,boost,enhanc,function,wide,prevent,parliamentari,pleasur,financ,oversea,oil,modern,oblig,coalit,judgment,combin,conserv,behaviour,regard,peer,affect,nuclear,deploy,commerci,strengthen,aid,celebr,illeg,suspect,news,qualiti,keen,option,correct,code,aspect,internet,licenc,institut,fantast,behalf,write,effici,sometim,debt,mental,chamber,christchurch,advantag,threat,pedlar,passion,stafford,emerg,danger,expand,consist,treatment,premis,ncs,appreci,capac,weapon,tabl,westminst,motion,everybodi,instanc,elsewher,conclus,broad,treasuri,manufactur,ultim,wider,suppli,target,express,brexit,sustain,charter,intellig,champion,screen,payment,restrict,immigr,maximum,construct,invit,search,consent,draw,manifesto,content,occas,lesson,particip,robust,damag,enorm,leadership,politician,convict,primari,ofcom,retain,transport,frank,type,slight,critic,deliveri,fundament,russia,normal,locat,crown,histor,honour,appal,transit,bureaucraci,convers,anim,condit,mess,somerset,snp,agenda,equal,status,medicin,necessarili,yesterday,incom,demonstr,knowledg,sad,profession,mechan,bench,privileg,desir,clinic,size,stick,altern,societi,royal,alongsid,attend,beauti,technic,music,permiss,commonwealth,corpor,ongo,excit,abus,background,alleg,reduct,legitim,easier,youth,fee,resourc,tool,choos,prosper,facil,expertis,equip,supermarket,afghanistan,valuabl,mayor,crucial,villag,dorset,onshor,spoken,borough,bear,otherwis,renew,somme
0,Adam Afriyie,Con,"[scienc, technolog, facil, grant, fellowship, ...",0.046602,0.010680,0.053398,0.001942,0.001942,0.003883,0.005825,0.005825,0.003883,0.014563,0.009709,0.002913,0.004854,0.009709,0.001942,0.001942,0.009709,0.005825,0.004854,0.002913,0.001942,0.001942,0.004854,0.001942,0.000971,0.001942,0.002913,0.000000,0.010680,0.003883,0.00

In [238]:
df.loc[df['party'] == 'Con', 'party_bool'] = 1
df.loc[df['party'] == 'Lab', 'party_bool'] = 0

/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_14956/3670770691.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[df['party'] == 'Con', 'party_bool'] = 1


# Regressions linéaires

In [239]:
import statsmodels.api as sm

dico={}
dico['Speaker'] = ['intercept', 'slope_coefficient']

for word in lili:
    x = df['party_bool']
    y = df[f'{word}']

    # adding the constant term
    x = sm.add_constant(x)

    result = sm.OLS(y, x).fit()

    dico[f'{word}'] = [result.params[0], result.params[1]]


In [240]:
aux = pd.DataFrame(dico)
df = pd.concat([df, aux]).reset_index(drop=True)

In [241]:
df.drop(columns=['somme'], inplace=True)
df = df[['Speaker', 'party', 'text']+['party_bool'] + list(df.columns)[3:-1]]

In [242]:
df

,Speaker,party,text,party_bool,hon,gentleman,govern,ladi,absolut,european,labour,agre,deliv,constitu,uk,commit,â£,recognis,money,union,debat,welcom,fund,amend,discuss,improv,nhs,kingdom,protect,law,secur,reform,consid,encourag,requir,benefit,eu,lord,bbc,job,sector,parliament,duti,oper,concern,review,technolog,legisl,regul,vote,effect,spend,measur,introduc,data,provis,inde,appli,britain,offenc,specif,addit,programm,propos,individu,wish,success,elect,claus,econom,congratul,obvious,shall,grate,inform,learn,progress,patient,enabl,agreement,proper,subject,awar,refer,england,therefor,organis,polit,independ,achiev,complet,court,respect,northern,ireland,excel,approach,exist,vital,delight,apprenticeship,shadow,sort,activ,practic,colleagu,perhap,g,exact,speaker,crime,beer,educ,extrem,scotland,commend,employ,hear,mention,negoti,assur,campaign,referendum,promot,suggest,yes,carri,polici,design,contribut,tribut,necessari,reassur,crimin,extra,serv,rural,fair,appropri,accept,arrang,scheme,greater,pub,enterpris,dog,innov,engin,scottish,control,chariti,fulli,respond,engag,product,conduct,listen,basi,trust,initi,reduc,common,train,scienc,deficit,welfar,confid,commiss,defenc,regim,maintain,investig,strategi,circumst,consum,standard,principl,consult,purpos,ministeri,rememb,prosecut,determin,object,mobil,argument,broadband,consider,longterm,midland,presid,speech,figur,resid,proud,proceed,generat,establish,border,coverag,cover,gas,skill,environ,export,unemploy,prison,repres,reflect,wind,incred,infrastructur,implement,safe,abil,green,relev,sentenc,cornwal,highlight,solut,taxpay,natur,competit,remind,answer,balanc,freedom,framework,pass,citizen,extend,simpli,regist,syria,attract,flexibl,draft,partnership,intervent,remov,sir,advic,rout,produc,remark,connect,communic,appoint,tackl,network,identifi,judg,perfect,penalti,intent,intend,link,commission,prioriti,chancellor,arm,cultur,drug,wale,applic,electr,word,section,outcom,custom,justic,inquiri,enforc,essenti,procedur,stalk,owner,price,statutori,treati,aim,terrorist,throughout,cancer,send,transpar,guidanc,sensibl,devolv,doubt,highway,recommend,choic,notic,urg,whom,approv,vehicl,regular,prepar,paper,elector,exercis,assist,strike,enjoy,inherit,safeguard,precis,sit,capabl,save,constitut,pupil,partner,treat,boost,enhanc,function,wide,prevent,parliamentari,pleasur,financ,oversea,oil,modern,oblig,coalit,judgment,combin,conserv,behaviour,regard,peer,affect,nuclear,deploy,commerci,strengthen,aid,celebr,illeg,suspect,news,qualiti,keen,option,correct,code,aspect,internet,licenc,institut,fantast,behalf,write,effici,sometim,debt,mental,chamber,christchurch,advantag,threat,pedlar,passion,stafford,emerg,danger,expand,consist,treatment,premis,ncs,appreci,capac,weapon,tabl,westminst,motion,everybodi,instanc,elsewher,conclus,broad,treasuri,manufactur,ultim,wider,suppli,target,express,brexit,sustain,charter,intellig,champion,screen,payment,restrict,immigr,maximum,construct,invit,search,consent,draw,manifesto,content,occas,lesson,particip,robust,damag,enorm,leadership,politician,convict,primari,ofcom,retain,transport,frank,type,slight,critic,deliveri,fundament,russia,normal,locat,crown,histor,honour,appal,transit,bureaucraci,convers,anim,condit,mess,somerset,snp,agenda,equal,status,medicin,necessarili,yesterday,incom,demonstr,knowledg,sad,profession,mechan,bench,privileg,desir,clinic,size,stick,altern,societi,royal,alongsid,attend,beauti,technic,music,permiss,commonwealth,corpor,ongo,excit,abus,background,alleg,reduct,legitim,easier,youth,fee,resourc,tool,choos,prosper,facil,expertis,equip,supermarket,afghanistan,valuabl,mayor,crucial,villag,dorset,onshor,spoken,borough,bear,otherwis,renew
0,Adam Afriyie,Con,"[scienc, technolog, facil, grant, fellowship, ...",1.0,0.046602,0.010680,0.053398,0.001942,0.001942,0.003883,0.005825,0.005825,0.003883,0.014563,0.009709,0.002913,0.004854,0.009709,0.001942,0.001942,0.009709,0.005825,0.004854,0.002913,0.001942,0.001942,0.004854,0.001942,0.000971,0.001942,0.002913,0.000000,0.010680,0.00

In [244]:
df.to_csv('01. output/df_500_with_slope_coef.csv')